In [ ]:
import os
import uuid
import json
import docker
from dulwich.repo import Repo
import time
import nbconvert
import nbformat
import random


In [ ]:
# set up Docker client and remote Docker Machine connection
tls_config = docker.tls.TLSConfig(
    client_cert=('wrnaz/cert.pem', 'wrnaz/key.pem'),
    verify='wrnaz/ca.pem'
)
client = docker.DockerClient(base_url='tcp://23.97.131.169:2376', tls=tls_config)


In [ ]:
sessions_path = '/home/jovyan/faas/faas_sessions/'
notebook = ''
logs = ''

def create_session():
    # create a session folder with a unique name
    id = str(uuid.uuid4())
    sdir = sessions_path+id
    if not os.path.exists(sdir):
        os.makedirs(sdir)
        return id
    else:
        return "TRAMIDAGIVIGA"


In [ ]:
#handle Jupyter notebook executions
#BUT DO IT FUCKING LATER WHEN THERE'S TIME!
def store_notebook(sid, fileName, notebook_json):
    filePathNameWExt = sessions_path + sid + '/' + fileName
    nb = nbformat.reads(str(notebook_json), as_version=4)
    nbformat.write(fp=filePathNameWExt, nb=nb)
    return filePathNameWExt


# run a Docker container executing user's notebook with
# Jupyter Kernel Gateway. Just to expose user defined API endpoints
# that can execute the code in the notebook itself or anything else..
def run_nb_on_kernelgateway(sid, nb_path):
    port  = str(random.randint(7000, 8000))
    ct = client.containers.run(image='wui_faas_session', 
                                      entrypoint="tini -- jupyter-kernelgateway --KernelGatewayApp.api=notebook-http --KernelGatewayApp.ip=0.0.0.0",
                                      command=" --KernelGatewayApp.port=" + port + " --KernelGatewayApp.seed_uri="+nb_path,
                                      #command="ls "+nb_path,
                                      labels={"faas_session_id": sid},
                                      ports={port+'/tcp': port},
                                      name="faas_"+sid,
                                      volumes_from="wui_faas_1",
                                      environment=["FAAS_SID=" + sid],
                                      detach=True)
    #ct = client.containers.run(image='wui_faas_session', command="ls /home/jovyan/", volumes_from="5d0abcd9dd5c750b063368f3957")
    if ct:
        return 'http://23.97.131.169:'+port+'/_api/spec/swagger.json'
    else:
        return 'ERROR: running container'
    

    


In [ ]:
# handle Python script execution
def store_script(path, fileName, data):
    filePathNameWExt = path + '/' + fileName
    print(filePathNameWExt)
    with open(filePathNameWExt, 'w+') as fp:
        fp.write(data)
    fp.close()
    return fp

def run_script(session_id):
    container = client.containers.run('jupyter/scipy-notebook', 'jupyter notebook '+ sessions_path + session_id + '/nb.ipynb')
    return container.id

def with_jupyter(notebook, return_format='STDOUT'):
    ct = client.containers.run('jupyter/scipy-notebook', 'jupyter-nbconvert --to notebook --execute foo.ipynb')
    return ct

In [ ]:
# prepare the JSON schema (or metadata) so we could make it as easy as possible
# but also not forget anything important our users would miss. 

# REMEMBER! this is currently only to send our notebooks, 
# scripts, apps, to the Machine for execution
REQUEST = json.dumps({
    'body' : {}
})

In [ ]:
# POST /notebook
try:
    req = json.loads(REQUEST)
    notebook_json = req['body']
    sid = create_session()
    nb_path = store_notebook(sid, 'nb.ipynb', notebook_json)
    if os.path.exists(nb_path):
        ct = run_nb_on_kernelgateway(sid, nb_path)
    else:
        print('ERROR: session folder missing')
except Exception as e:
    response = e
finally:
    print(str(ct))


In [ ]:
# POST /ipynb
request = json.loads(REQUEST)
notebook_json = request['body']
print(notebook_json)

In [ ]:
# GET /sessions
for c in client.containers.list(all=True):
    print(c)